In [46]:
from matplotlib.pylab import rcParams
import xlrd
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.4f' % x)
import seaborn as sns
sns.set_context("paper", font_scale=1.3)
sns.set_style('white')
import warnings
warnings.filterwarnings('ignore')
from time import time
import matplotlib.ticker as tkr
from scipy import stats
from statsmodels.tsa.stattools import adfuller
from sklearn import preprocessing
from statsmodels.tsa.stattools import pacf
%matplotlib inline
import math
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from keras.callbacks import EarlyStopping

from keras.preprocessing.sequence import TimeseriesGenerator
from datetime import datetime

In [47]:
rcParams["figure.figsize"] = 20,20

def parser(x):
    return datetime.strptime(x, "%m/%d/%y")

df = pd.read_excel(
     os.path.join("test.xlsx"),sheet_name="Data", engine='openpyxl', parse_dates=[0], date_parser = parser
)


In [48]:
df.drop(["Severity",'Case ID', "Status","Total" ],axis = 1, inplace =True)
df = df.groupby(["Entered on"], as_index = False).mean()
df = df.rename(columns={'Resolution Time': 'value'})
df = df.fillna(0)


In [49]:
df

,Entered on,value
0,2016-09-12,31.0000
1,2016-09-14,23.5000
2,2016-09-20,8.0000
3,2016-09-30,5.0000
4,2016-10-07,19.0000
...,...,...
788,2021-07-26,3.0000
789,2021-07-27,0.0000
790,2021-07-28,0.0000
791,2021-07-29,0.0000


In [50]:

close_data = df['value'].values
close_data = close_data.reshape((-1,1))

split_percent = 0.80
split = int(split_percent*len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]

date_train = df['Entered on'][:split]
date_test = df['Entered on'][split:]

print(len(close_train))
print(len(close_test))

634
159


In [51]:
close_data.shape

(793, 1)

In [52]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler

def get_scaler(scaler):
    scalers = {
        "minmax": MinMaxScaler,
        "standard": StandardScaler,
        "maxabs": MaxAbsScaler,
        "robust": RobustScaler,
    }
    return scalers.get(scaler.lower())()
# scaler = get_scaler('minmax')
# close_train = scaler.fit_transform(close_train)
# close_test = scaler.transform(close_test)


In [58]:
look_back =  10

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=32)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

In [74]:

from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(
    LSTM(70,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 350
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)

Epoch 1/350
20/20 [==============================] - 3s 151ms/step - loss: 754.2943
Epoch 2/350
20/20 [==============================] - 0s 15ms/step - loss: 589.7591
Epoch 3/350
20/20 [==============================] - 0s 17ms/step - loss: 588.1445
Epoch 4/350
20/20 [==============================] - 0s 14ms/step - loss: 567.2295
Epoch 5/350
20/20 [==============================] - 0s 16ms/step - loss: 572.5171
Epoch 6/350
20/20 [==============================] - 0s 15ms/step - loss: 562.8043
Epoch 7/350
20/20 [==============================] - 0s 15ms/step - loss: 557.4478
Epoch 8/350
20/20 [==============================] - 0s 17ms/step - loss: 553.5898
Epoch 9/350
20/20 [==============================] - 0s 17ms/step - loss: 550.6368
Epoch 10/350
20/20 [==============================] - 0s 16ms/step - loss: 546.7748
Epoch 11/350
20/20 [==============================] - 0s 16ms/step - loss: 559.0005
Epoch 12/350
20/20 [==============================] - 0s 18ms/step - loss: 549.3111


Epoch 195/350
20/20 [==============================] - 0s 14ms/step - loss: 62.2746
Epoch 196/350
20/20 [==============================] - 0s 16ms/step - loss: 50.5681
Epoch 197/350
20/20 [==============================] - 0s 16ms/step - loss: 42.9723
Epoch 198/350
20/20 [==============================] - 0s 17ms/step - loss: 35.3762
Epoch 199/350
20/20 [==============================] - 0s 19ms/step - loss: 31.4320
Epoch 200/350
20/20 [==============================] - 0s 15ms/step - loss: 28.9655
Epoch 201/350
20/20 [==============================] - 0s 14ms/step - loss: 27.2154
Epoch 202/350
20/20 [==============================] - 0s 15ms/step - loss: 24.1182
Epoch 203/350
20/20 [==============================] - 0s 16ms/step - loss: 23.3988
Epoch 204/350
20/20 [==============================] - 0s 15ms/step - loss: 21.5428
Epoch 205/350
20/20 [==============================] - 0s 14ms/step - loss: 20.0575
Epoch 206/350
20/20 [==============================] - 0s 14ms/step - loss: 

In [75]:
prediction = model.predict_generator(test_generator)


# close_train_arr = scaler.inverse_transform(close_train)
# close_test_arr = scaler.inverse_transform(close_test)
# prediction_arr = scaler.inverse_transform(prediction)

data_train = close_train.reshape((-1))
data_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

In [90]:
import plotly.offline as pyo
import plotly.graph_objs as go
from plotly.offline import iplot

trace1 = go.Scatter(
    x = date_train,
    y = data_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = data_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Resolution Time Prediction",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

In [111]:
close_data = close_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df['Entered on'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 64
forecast_value = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

In [141]:
series = pd.DataFrame(forecast_dates, columns= ["Entered on"])
forecast1 = pd.DataFrame(forecast_value, columns=['value'])# change here to future if u want them seperate
df_f = pd.concat([series, forecast1], axis=1)
df_f = df_f.set_index(df_f["Entered on"])
future = df_f.drop(columns = ["Entered on"])
df_new = df.set_index(["Entered on"])
result = pd.concat([df_new, future], axis=0)

sums = result.select_dtypes(pd.np.number).sum().rename('Total')
# value, forecast = np.sum(result, axis = 0)
result.append(sums)

,value
Entered on,
2016-09-12 00:00:00,31.0000
2016-09-14 00:00:00,23.5000
2016-09-20 00:00:00,8.0000
2016-09-30 00:00:00,5.0000
2016-10-07 00:00:00,19.0000
...,...
2021-09-29 00:00:00,21.1568
2021-09-30 00:00:00,24.0554
2021-10-01 00:00:00,18.7084


In [89]:
import plotly.offline as pyo
import plotly.graph_objs as go
from plotly.offline import iplot



trace1 = go.Scatter(
    x = df["Entered on"],
    y = close_data,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = forecast_dates,
    y = forecast,
    mode = 'lines',
    name = 'Forecast'
)
trace3 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)

layout = go.Layout(
    title = "Forecast Resolution Time",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Average Time"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()